In [ ]:
# Setup and imports
import os
import sys
import json
from pathlib import Path
from typing import List
from pydantic import BaseModel, Field

# Add project root to path
project_root = Path('.').resolve().parent.parent
sys.path.insert(0, str(project_root))

# Import our unified utility
from langgraph_utils import create_agent, ExampleAgentOutput, ChainOfThoughtOutput

# Try to import LangChain models
try:
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-5-haiku-latest", temperature=0.1)
    print("✅ LangChain LLM available")
except ImportError:
    llm = None
    print("⚠️ LangChain not available - using mock mode")

# Define custom schema for testing
class HealthcareResponse(BaseModel):
    """Healthcare assistant response schema"""
    response: str = Field(description="Main response to user")
    urgency: str = Field(description="Urgency level: low, medium, high")
    next_steps: List[str] = Field(description="Recommended next steps")
    confidence: float = Field(description="Confidence score", ge=0.0, le=1.0)

print("🎛️ Setup complete - ready to test create_agent() patterns")


In [ ]:
# 1. LangChain structured output (recommended pattern)
print("=== 🏆 LangChain Structured Pattern ===")

healthcare_agent = create_agent(
    name="HealthcareAssistant",
    prompt_path="example_prompt.md",
    examples_path="example_examples.json",
    output_schema=HealthcareResponse,
    llm=llm,
    # LangChain defaults (all True)
    use_langchain_pattern=True,
    use_structured_output=True,
    use_human_message=True,
    use_system_message=True,
    merge_examples=True
)

print(f"✅ Created: {healthcare_agent.__name__}")
print(f"📖 Documentation: {healthcare_agent.__doc__[:200]}...")

# Test with healthcare query
result = healthcare_agent("I need to find a cardiologist who takes my insurance")
print(f"\n🎯 Result type: {type(result)}")
print(f"📝 Response: {result.response[:100]}...")
print(f"⚡ Urgency: {result.urgency}")
print(f"📋 Next steps: {len(result.next_steps)} items")
print(f"🎯 Confidence: {result.confidence}")

print("\n✅ LangChain pattern returns Pydantic objects directly!")


In [ ]:
# 2. Test different message patterns
print("=== 🔧 Message Pattern Variations ===")

# 2a. SystemMessage + HumanMessage (default)
print("\n📨 Pattern A: SystemMessage + HumanMessage")
agent_a = create_agent(
    name="MessagePatternA",
    custom_system_message="You are a helpful healthcare assistant.",
    output_schema=ExampleAgentOutput,
    use_system_message=True,
    use_human_message=True,
    use_mock_mode=True
)

result_a = agent_a("Find me a doctor")
print(f"Result: {type(result_a)} - {result_a.response[:60]}...")

# 2b. Combined message (no HumanMessage)
print("\n📨 Pattern B: Combined Message (no HumanMessage)")
agent_b = create_agent(
    name="MessagePatternB",
    custom_system_message="You are a helpful healthcare assistant.",
    output_schema=ExampleAgentOutput,
    use_system_message=True,
    use_human_message=False,  # Input appended to system message
    use_mock_mode=True
)

result_b = agent_b("Find me a doctor")
print(f"Result: {type(result_b)} - {result_b.response[:60]}...")

# 2c. Single prompt (no SystemMessage)
print("\n📨 Pattern C: Single Prompt (no SystemMessage)")
agent_c = create_agent(
    name="MessagePatternC",
    custom_system_message="You are a helpful healthcare assistant.",
    output_schema=ExampleAgentOutput,
    use_system_message=False,  # Everything in one message
    use_human_message=True,
    use_mock_mode=True
)

result_c = agent_c("Find me a doctor")
print(f"Result: {type(result_c)} - {result_c.response[:60]}...")

print("\n✅ All message patterns work with same interface!")
